In [ ]:
from models.Pix2Pix import Pix2PixGAN
from utils.Dataset import *
from utils.lossTracker import save_losses, load_losses
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
print(f'Using {device}')
torch.cuda.empty_cache() if torch.cuda.is_available() else torch.mps.empty_cache() if torch.mps.is_available() else None
model = Pix2PixGAN(device, max_epochs=400, start_decay_epoch=200)

load_model = False
if load_model:
    start_epoch = model.load_state('checkpoint_epoch_400', device)
    losses_dict = load_losses()
    if losses_dict:
        g_loss = losses_dict['g_loss']
        d_loss = losses_dict['d_loss']
    g_loss = []
    d_loss = []
else:
    start_epoch = 0
    g_loss = []
    d_loss = []
end_epoch = 399

torch.backends.cudnn.benchmark = True

for epoch in range(start_epoch, end_epoch):
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{end_epoch}", leave=False) as pbar:
        for real_A, real_B in pbar:
            loss_D, loss_G = model.train_step(real_A, real_B)
            pbar.set_postfix({
                "Loss D": loss_D,
                "Loss G": loss_G,
                "LR D": model.optimizer_D.param_groups[0]['lr'],
                "LR G": model.optimizer_G.param_groups[0]['lr']
            })

        # Адаптация скорости обучения
        model.step_schedulers()

        g_loss.append(loss_G)
        d_loss.append(loss_D)

        if (epoch + 1) % 1 == 0:
            model.save_state(epoch)
            save_losses(g_loss=g_loss, d_loss=d_loss)


Using cuda


Epoch 1/4:   0%|          | 0/182 [00:00<?, ?it/s]

Epoch 2/4:   0%|          | 0/182 [00:00<?, ?it/s]